<a href="https://colab.research.google.com/github/DreamTeamInc/Core-DS/blob/issues%2F9/models/day/ModelDeploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подключение GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Path to 'sample_core_analysis_data_example' folder with folder name
# SAMPLE_CORE_PATH = 'drive/My Drive/ sample_core_analysis_data_example'
PHOTO_PREDICT_PATH = 'drive/My Drive/photo_predict/'
MASK_PREDICT_PATH = 'drive/My Drive/mask_predict/'
MODEL_PATH = 'drive/My Drive/models/'
# CLASSIFICATION_PATH = 'drive/My Drive/classfications/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Подключение библиотек для функций считывания файлов

In [ ]:
import pandas as pd
import numpy as np
import json

from skimage import io

pd.options.mode.chained_assignment = None

In [ ]:
# !pip install pytorch-lightning
# !pip install imgaug==0.4.0

In [ ]:
def get_photo_df(data, i):
    return io.imread(SAMPLE_CORE_PATH + '/photos/{}.jpeg'.format(data['photo_id'].iloc[i]))

def get_mask_df(data, i):
    return np.load(SAMPLE_CORE_PATH + '/matrixes/matrix_{}__{}.npz'.format(data['photo_id'].iloc[i], data['task_id'].iloc[i]))['data']

def get_json_df(data, i):
    datastore = ''
    with open(SAMPLE_CORE_PATH + '/extra/extra_{}__{}.json'.format(data['photo_id'].iloc[i],
                                                                   data['task_id'].iloc[i]), 'r', encoding="utf8") as f:
        datastore = json.load(f)

    return datastore

In [ ]:
import torch
from torchvision import models
import requests
import json
import os
import skimage.io as io
import imgaug.augmenters as iaa
from skimage import img_as_float32

class DayModel:
    def __init__(self, model_path, num_classes):
        self.model = models.segmentation.deeplabv3_resnet50(pretrained=False, progress=True, num_classes=num_classes)
        self.model.load_state_dict(torch.load(model_path))
        self.is_cuda_available = torch.cuda.is_available()
        # Set model to cuda if available
        if self.is_cuda_available:
            self.model.cuda()


    def predict(self, photo_path):
        # Read photo
        photo = self.__get_image(photo_path)
        # Initialize augmentators
        aug_in = iaa.Resize({"height": 3000, "width": 300}, interpolation='linear')
        aug_out = iaa.Resize({"height": photo.shape[0], "width": photo.shape[1]}, interpolation='nearest')
        # aug_in applying
        photo = np.expand_dims(photo, axis=0)
        photo = aug_in(images=photo)[0]
        # Check if photo allowed
        self.__is_photo_allowed(photo)
        # Convert photo to torch.Tensor
        photo = self.__prepare_photo(photo)
        # Make predict
        self.model.eval()
        mask_raw = self.model(photo)
        mask = torch.max(mask_raw['out'], 1)[1].cpu().numpy()[0].astype(np.uint8)
        print(np.unique(mask))
        # aug_out applying
        mask = np.expand_dims(mask, axis=0)
        mask = aug_in(images=mask)[0]
        self.__send_mask(mask, photo_path.replace(PHOTO_PREDICT_PATH, '').replace('photo', '').replace('.jpg', '').replace('.png', '').replace('.jpeg', ''))


    def fit(self):
        pass
    

    def __is_photo_allowed(self, photo):
        if len(photo.shape) != 3:
            raise ValueError('Except 3 dimensional photo, got {}'.format(photo.shape))
        if photo.shape[2] != 3:
            raise ValueError('Except 3-channel image, got {}'.format(photo.shape[2]))
        return True


    def __prepare_photo(self, photo):
        photo = img_as_float32(photo)
        photo = photo.transpose((2, 0, 1))
        photo = torch.from_numpy(photo.copy()).unsqueeze(0).float()
        if self.is_cuda_available:
            photo = photo.cuda()
        return photo
    

    def __get_image(self, image_path):
        return io.imread(image_path)


    def __save_image(self, fname, image):
        return io.imsave(fname, image)


    def __send_mask(self, mask, id):
        fname = MASK_PREDICT_PATH + 'mask' + id + '.png'
        self.__save_image(fname, mask)
        url = 'https://code-backend.herokuapp.com/send_mask/'
        # url = 'http://185.243.215.18:8080/send_mask/'
        files = {'mask': (os.path.basename(fname), open(fname, 'rb'),
                          'multipart/formdata')}
        print(requests.post(url, files=files).text)

In [ ]:
import datetime
import time
import re

def wait_task():
    while True:
        if os.listdir(PHOTO_PREDICT_PATH):
            try:
                photo_name = os.listdir(PHOTO_PREDICT_PATH)[0]
                io.imread(PHOTO_PREDICT_PATH + photo_name)
            except:
                continue
            print(photo_name)
            # user_id =  re.findall('[\d]+', photo_name)[1]
            # model_id = re.findall('[\d]+', photo_name)[2]
            # fname = MODEL_PATH + 'model' + user_id + '_' + model_id + '.pt'
            fname = MODEL_PATH + 'model5_1.pt'
            model = DayModel(fname, 4)  # QUANTITY OF SEGMENT TYPES IS HARDCODED
            model.predict(PHOTO_PREDICT_PATH + photo_name)
            print(datetime.datetime.now())
            os.remove(PHOTO_PREDICT_PATH + photo_name)
        time.sleep(5)

In [ ]:
wait_task()

photo53_44_63.png


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



[2 3]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: UserWarning: drive/My Drive/mask_predict/mask53_44_63.png is a low contrast image


{"message":"mask is successfully sent"}
2020-11-25 07:03:11.521063
